# Parent Document Retriever

When splitting documents for retrieval, there are often conflicting desires:

    You may want to have small documents, so that their embeddings can most accurately reflect their meaning. If too long, then the embeddings can lose meaning.
    You want to have long enough documents that the context of each chunk is retained.

The ParentDocumentRetriever strikes that balance by splitting and storing small chunks of data. During retrieval, it first fetches the small chunks but then looks up the parent ids for those chunks and returns those larger documents.

Note that "parent document" refers to the document that a small chunk originated from. This can either be the whole raw document OR a larger chunk.

In [1]:
from langchain.retrievers import ParentDocumentRetriever

from langchain.document_loaders import TextLoader
# from langchain.embeddings import OpenAIEmbeddings
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name = "moka-ai/m3e-base")

d:\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loaders = [
    TextLoader("../../docs/paul_graham_essay.txt"),
    TextLoader("../../docs/state_of_the_union.txt"),
]
docs = []
for l in loaders:
    docs.extend(l.load())

# Retrieving full documents
In this mode, we want to retrieve the full documents. Therefore, we only specify a child splitter.

In [3]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding_model
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [4]:
retriever.add_documents(docs, ids=None)

This should yield two keys, because we added two documents.

In [5]:
list(store.yield_keys())

['7efa16ff-ded1-4877-90c8-900c74de940f',
 '1ab3e742-90c2-4c03-8db3-8a22539d9f36']

Let's now call the vector store search functionality - we should see that it returns small chunks (since we're storing the small chunks).

In [6]:
sub_docs = vectorstore.similarity_search("justice breyer")

In [7]:
print(sub_docs[0].page_content)

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.


Let's now retrieve from the overall retriever. This should return large documents - since it returns the documents where the smaller chunks are located.

In [8]:
retrieved_docs = retriever.get_relevant_documents("justice breyer")

In [9]:
len(retrieved_docs[0].page_content)

38540

## Retrieving larger chunks
Sometimes, the full documents can be too big to want to retrieve them as is. In that case, what we really want to do is to first split the raw documents into larger chunks, and then split it into smaller chunks. We then index the smaller chunks, but on retrieval we retrieve the larger chunks (but still not the full documents).

In [10]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=embedding_model
)
# The storage layer for the parent documents
store = InMemoryStore()

In [11]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [12]:
retriever.add_documents(docs)

We can see that there are much more than two documents now - these are the larger chunks.

In [13]:
len(list(store.yield_keys()))

66

Let's make sure the underlying vector store still retrieves the small chunks.

In [14]:
sub_docs = vectorstore.similarity_search("justice breyer")

In [15]:
print(sub_docs[0].page_content)

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.


In [16]:
retrieved_docs = retriever.get_relevant_documents("justice breyer")

In [17]:
len(retrieved_docs[0].page_content)

1849

In [18]:
print(retrieved_docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal publi